# Imports

In [89]:
from ultralytics import YOLO
import os
from scipy.io import loadmat
import numpy as np

In [101]:
# load model
model = YOLO("yolov8m-pose.pt")

# configurations
N = 10
MPII_IMAGES_DIR = "./mpii/images"
IMAGE_FILES = [os.path.join(MPII_IMAGES_DIR, file) for file in os.listdir(MPII_IMAGES_DIR) if file.endswith(('.jpg'))][:N]


results_list = []
# iterate for each image
for image_file in IMAGE_FILES:
    # execute
    results = model(source=image_file, show=False, conf=0.3, save=False)
    results_list.append(results)
del model

# USE ANNOTATION MPII

annotations_data = loadmat("./mpii/annotations/mpii_human_pose.mat")
# extract testing annotation list structure from the entire annotation list
annolist_test = annotations_data['RELEASE'] ['annolist']
#print("Data Structure Annolist Test", annolist_test) 
# annolist_test  two-dimensional array of structure


# extract groups of people
# here I extract groups of people using the getMultiPersonGroups function from the MPII assessment toolset. This is done by loading the groups from a previously generated file.
data = loadmat('./mpii/eval/groups_v12.mat')
groups = data["groups"]

# Load and use getMultiPersonGroups function
def getMultiPersonGroups(groups, release_data):
    imgidxs_multi_test = []
    rectidxs_multi_test = []
    for group in groups:
        imgidxs = []
        rectidxs = []
        for item in group:
            if len(item) == 0:  # Check if item is empthy
                continue
            if len(item[0]) > 0:
                imgidxs.append(item[0][0])
            if len(item[0]) > 1:
                rectidxs.append(item[0][1])
            else:
                rectidxs.append([])
        imgidxs_multi_test.append(imgidxs)
        rectidxs_multi_test.append(rectidxs)
    return imgidxs_multi_test, rectidxs_multi_test
    
#print("imgidxs_multi_test", imgidxs_multi_test)

imgidxs_multi_test, rectidxs_multi_test = getMultiPersonGroups(groups, annotations_data['RELEASE'])

# Prepare predictions
pred = []
########################################################################

# Verificar e achatar a estrutura de imgidxs_multi_test
print("actual structure of imgidxs_multi_test:")
print(type(imgidxs_multi_test))  # Deve ser list
print(type(imgidxs_multi_test[0]))  # Deve ser list

# Se imgidxs_multi_test for uma lista aninhada, você pode desaninhá-la usando a função de achatamento
imgidxs_multi_test_flat = [item for sublist in imgidxs_multi_test for item in sublist]

# Verifique novamente a estrutura após o achatamento
print("\nimgidxs_multi_test actual structure:")
print(type(imgidxs_multi_test_flat))  # Deve ser list

# Atualizar imgidxs_multi_test para ser a versão achatada
imgidxs_multi_test = imgidxs_multi_test_flat

#######################################################################
# perform pre-processing for each group
for imgidxs, rects in zip(imgidxs_multi_test, rectidxs_multi_test):
    for imgidx, ridxs in zip(imgidxs, rects):
        if imgidx >= len(annolist_test):
            print("Error: imgidx value is outside limits of annolist_test")
            continue

        for ridx in ridxs:
            if len(annolist_test[imgidx][0]) <= ridx:
                print("Error: ridx  value is outside limits of annolist_test[imgidx][0]")
                continue
            
            rect = annolist_test[imgidx][0][ridx][0][0]
            # initialize variables to compute group boundaries
            x1_group, y1_group = np.inf, np.inf
            x2_group, y2_group = -np.inf, -np.inf
            scale_sum = 0


    
    # Compute group boundaries and scale
    for imgidx, ridxs in zip(imgidxs, rects):
        for ridx in ridxs:
            print("Valor de imgidx:", imgidx)
            print("imgidx:", imgidx)
            print("ridx:", ridx)
            rect = annolist_test[imgidx][0][ridx][0][0]
            # Check if the number of people in the group is greater than zero
            num_people = len(rects)
            if num_people > 0:
                x1_group = min(x1_group, rect['objpos'][0][0][0][0])
                y1_group = min(y1_group, rect['objpos'][0][0][1][0])
                x2_group = max(x2_group, rect['objpos'][0][0][0][0])
                y2_group = max(y2_group, rect['objpos'][0][0][1][0])
                scale_sum += rect['scale'][0][0][0]
            else:
                print("Warning: Skipping group with no people")
                continue  # Skip this group and continue with the next one
    
    # Compute average scale of the group
    num_people = len(rects)
    if num_people > 0:
        scale_group = scale_sum / num_people
    else:
        scale_group = 0


image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000001163.jpg: 384x640 2 persons, 626.4ms
Speed: 8.0ms preprocess, 626.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000003072.jpg: 384x640 3 persons, 616.4ms
Speed: 0.0ms preprocess, 616.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000004812.jpg: 384x640 1 person, 588.0ms
Speed: 0.0ms preprocess, 588.0ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\OneDrive\Documentos\Code\MediaPipe-MPII-and-YOLO-MPII-comparison\mpii\images\000005283.jpg: 384x640 2 persons, 683.8ms
Speed: 0.0ms preprocess, 683.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\bp111\On

IndexError: index 1 is out of bounds for axis 0 with size 1